# 深度学习 实验3

本节课我们实现包含隐层的神经网络模型，并在手写数字识别数据集上进行测试。

## 数据集

手写数字识别数据集

下载链接：https://www.kaggle.com/c/digit-recognizer/data

（本节课仅需要训练集）

## 代码说明

下面的代码构造了一个包含隐层的神经网络模型，模型分为输入层、隐含层、输出层

输入层包含 $784$ 个神经元，输入手写数字的灰度向量，输出 $U$
$$
U=X
$$

隐含层包含 $30$ 个神经元，输入 $U$，输出 $V$
$$
V=\sigma(W_1U)
$$

输出层包含 $10$ 个神经元，输入 $V$，输出 $\hat Y$ 表示每个数字的概率
$$
\hat Y=\sigma(W_2V)
$$

其中 $\sigma$ 表示 Sigmoid 函数，每个样本数据都以列向量进行存储和计算，损失函数取所有样本预测值的均方误差

$$
Loss=\frac{1}{N}\sum_{i=1}^N (\hat Y_i-Y_i)^2
$$

但是，这个模型的效率太低了，请做出适当修改（**或删了重写**），以提高训练效率和准确率。

## 可选的修改方式

### 1. 向模型中添加阈值

$$
U=X+B_0
$$
$$
V=\sigma(W_1U+B_1)
$$
$$
\hat Y=\sigma(W_2V+B_2)
$$

（在这个模型中加不加阈值影响不大。。。）
### 2. 改为随机梯度下降

代码中每次迭代会计算所有样本的均方误差，这是极其低效的，可以修改为每次取一批样本（例如128个）计算损失和梯度并更新参数。

### 3. 修改激活函数与损失函数

手写数字识别分类问题本质是多分类问题，对于多分类问题，一个常用的组合是 Softmax 函数与交叉熵损失，这可以提高模型的收敛速度。

### 4. 使用高效的优化算法

随机梯度下降法固然是一个高效的优化算法，但也可选择更高效的优化算法，例如 Adam 算法。

### 5. 调参

A：这就是你的深度学习系统？

B：是呀！你把数据倒进这一大堆线性代数里，再到另一边等着答案出来。

A：如果答案是错的呢？

B：那就拿根棍子搅一搅这堆东西，一直搅到答案正确未为止。

![](https://i.loli.net/2021/03/29/dTsolpvPxk92YXe.jpg)

In [1]:
import pandas as pd
import numpy as np


# 从 DataFrame 中读取x与y，转化为 numpy 数组
def get_x_y(data):
    x = data[[i for i in list(data.columns) if i != "label"]].values
    # 将标签转化为 one-hot 向量
    y_list = []
    for i in data["label"]:
        y = [0] * 10
        y[i] = 1
        y_list.append(y)
    return np.array(x) / 255, np.array(y_list)

data = pd.read_csv("train.csv")
train_x, train_y = get_x_y(data)

In [ ]:
# 计算准确率
def accuracy(y_pred, y_true):
    ac = 0
    for i in range(y_true.shape[1]):
        # 计算预测的标签
        temp = [y_pred[j][i] for j in range(10)]
        label_pred = temp.index(max(temp))
        # 计算真实的标签
        temp = [y_true[j][i] for j in range(10)]
        label_true = temp.index(max(temp))
        if label_pred == label_true:
            ac += 1
        # print(label_pred,label_true,temp)
    return ac / y_true.shape[1]


# 神经网络的层（啥都不做的层）
class DenseLayer:
    def __init__(self, n):
        self.n = n

    def connect(self, layer):
        pass

    def forward(self, x):
        return x

    def backward(self, dy):
        return dy


# 神经网络的层（全连接层）
class LinearLayer:
    # 初始化
    def __init__(self, n, activation, activation_diff):
        # 神经元个数
        self.n = n
        # 激活函数
        self.activation = activation
        # 激活函数导数
        self.activation_diff = activation_diff
        # 学习率
        self.eta = 0.001
        self.beta1 = 0.9
        self.beta2 = 0.999
        self.iter = 0
        self.mw = None
        self.vw = None
        self.mb = None
        self.vb = None

    # 全连接，随机初始化 w
    def connect(self, layer):
        self.w = np.random.rand(self.n, layer.n) * 0.2 - 0.1
        self.b = np.zeros((self.n, 1))
        self.mw = np.zeros_like(self.w)
        self.vw = np.zeros_like(self.w)
        self.mb = np.zeros_like(self.b)
        self.vb = np.zeros_like(self.b)

    # 前向传播
    def forward(self, x):
        self.x = x
        self.u = np.matmul(self.w, self.x) + self.b
        self.y = self.activation(self.u)
        return self.y

    # 反向传播 传入损失函数对该层每个输出值的导数，传出损失函数对该层每个输入值的导数
    def backward(self, dy, update=True):
        self.du = dy * self.activation_diff(self.u)
        self.db = self.du.mean(axis=1)
        self.dw = np.matmul(self.du, self.x.T)
        self.dx = np.matmul(self.w.T, self.du)
        if update:
            self.iter += 1
            self.mw = self.beta1 * self.mw + (1 - self.beta1) * self.dw
            self.vw = self.beta2 * self.vw + (1 - self.beta2) * (self.dw ** 2)
            self.mw_hat = self.mw / (1 - (self.beta1 ** self.iter))
            self.vw_hat = self.vw / (1 - (self.beta2 ** self.iter))
            self.vw_sqrt = np.sqrt(self.vw_hat)
            self.w -= self.eta * self.mw_hat / (self.vw_sqrt + 0.00000001)
            self.mb = self.beta1 * self.mb + ((1 - self.beta1) * self.db).reshape(self.n, 1)
            self.vb = self.beta2 * self.vb + ((1 - self.beta2) * (self.db ** 2)).reshape(self.n, 1)
            self.mb_hat = self.mb / (1 - (self.beta1 ** self.iter))
            self.vb_hat = self.vb / (1 - (self.beta2 ** self.iter))
            self.vb_sqrt = np.sqrt(self.vb_hat)
            self.b -= self.eta * self.mb_hat / (self.vb_sqrt + 0.00000001)
        return self.dx


# sigmoid 激活函数 f(x)=1/(1+exp(-x))
def sigmoid(x):
    return 1 / (1 + np.exp(-x))


# sigmoid 激活函数的导数 f'(x)=f(x)(1-f(x))
def sigmoid_diff(x):
    return sigmoid(x) * (1 - sigmoid(x))


# softmax 函数
def softmax(x):
    shift_x = x - np.max(x)
    exp_x = np.exp(shift_x)
    return exp_x / np.sum(exp_x)


# 无效激活函数
def invalid(x):
    return x


def invalid_diff(x):
    return 1


# 随机抽取指定数量的数据并转置
def random_choice(n, x, y):
    random_int = np.random.randint(0, len(x), n)
    mini_x = np.zeros((n, len(train_x[0])))
    mini_y = np.zeros((n, len(train_y[0])))
    j = 0
    for i in random_int:
        mini_x[j] = train_x[i]
        mini_y[j] = train_y[i]
        j += 1
    return mini_x.T, mini_y.T


# 构造神经网络
input_layer = DenseLayer(784)
hidden_layer = LinearLayer(30, sigmoid, sigmoid_diff)
hidden_layer.connect(input_layer)
output_layer = LinearLayer(10, invalid, invalid_diff)
output_layer.connect(hidden_layer)
n = 1000

for iteration in range(5000):
    # 正向传播
    mini_x, mini_y = random_choice(n, train_x, train_y)
    u = input_layer.forward(mini_x)
#     mini_x = train_x.T
#     mini_y = train_y.T
    v = hidden_layer.forward(u)
    y_hat = output_layer.forward(v)
    for i in range(n):
        y_hat[:, i] = softmax(y_hat[:, i])
    # 计算损失函数对每个输出值的导数
    # d_loss = (y_hat - mini_y) / y_hat.shape[1]
    d_loss_d_z = y_hat - mini_y
    # 反向传播
    dv = output_layer.backward(d_loss_d_z)
    du = hidden_layer.backward(dv)
    # 输出准确率
    if iteration % 100 == 0:
        print(accuracy(y_hat, mini_y))
u = input_layer.forward(train_x.T)
v = hidden_layer.forward(u)
y_hat = output_layer.forward(v)
for i in range(n):
    y_hat[:, i] = softmax(y_hat[:, i])
print("对于所有数据，准确度为{}".format(accuracy(y_hat, train_y.T)))


0.078
0.741
0.814
0.877
0.894
0.896
0.903
0.925
0.923
0.922
0.931
0.944
0.93
0.958
0.934
0.938
0.939
0.934
0.938
0.955
